# Boundaries and UFL in FEniCSx 1


## Practice Code for Boundary and UFL

Date: 22nd May 2023

In [ ]:
#please run twice
try:
    import dolfinx
except ImportError:
    !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/9b21f39/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

In [68]:
try:
    import viskex
except ImportError:
    !pip3 install "viskex@git+https://github.com/viskex/viskex.git@8895f22"
    import viskex


In [69]:
from ufl import *
import ufl
import numpy

from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import mesh, fem, io, nls, log

In [70]:
# the mesh creation
domain = mesh.create_unit_square(MPI.COMM_WORLD, 50, 50)
#the mesh is 2 dimensional unit square


UFL: Unified Form Language


---


# x : $\vec{r}$

dimension of x comes from the domain  



In [71]:
# this is r_vector
x = ufl.SpatialCoordinate(domain)

lets write a linear polynomial:

$u(x,y)=1+5x+3y$



In [72]:
U_exact=1+5*x[0]+3*x[1]

$u(x,y)=2+3x^2y^2+\sqrt(y)$

In [73]:

U_1=2+3*(x[0]**2)*(x[1]**2)+sqrt(x[1])

Let's check if it works

In [74]:
u_1 = lambda x: eval(str(U_1))

#lambda is a python thing to make functions, here it is telling to 
#i) convert UFL to string
#ii) Evaluate the string
#iii) Take x as input
#iv) u_exact is a function
print(u_1([1,2]))

15.414213562373096


# Reference fror UFL

## the page contains all the functions and expressions that can be used

https://fenicsproject.org/olddocs/ufl/1.5.0/ufl.html

# Best Reference: the documentation

https://buildmedia.readthedocs.org/media/pdf/fenics-ufl/latest/fenics-ufl.pdf

The in-built functions that are available:

max_value, min_value,

abs, sign, sqrt,

exp, ln, erf,

cos, sin, tan,

acos, asin, atan, atan_2,

cosh, sinh, tanh,

bessel_J, bessel_Y, bessel_I, bessel_K


#Example

$u(x,y)=1+x^2e^y+\sqrt(\sinh(xy))$

In [75]:
u_e=1+(x[0]**2)*exp(x[1])+sqrt(sinh(x[0]*x[1]))
u_f=lambda x: eval(str(u_e))
print(u_f([1,3]))

24.250639595678706


$u(x,y)=\ln(x^2+y^2)$

In [76]:
u_1=ln(x[0]**2+x[1]**2)
U1=lambda x: eval(str(u_1))
print(U1([1,0]))

0.0


## 3 dimensional expression

$u(x,y,z)=x^2y^3z^4$

In [77]:
trial_mesh= mesh.create_unit_cube(MPI.COMM_WORLD, 50, 50,50)
#3d geometry where we will work
x = ufl.SpatialCoordinate(trial_mesh)
#this is the vector for 3d geometry
U_t=(x[0]**2)*(x[1]**3)*(x[2]**4)
# this is the expression
Ut=lambda x: eval(str(U_t))
# this is the function
print(Ut([1,1,1]),'all ones')
print(Ut([2,1,1]),'x=2')
print(Ut([1,2,1]),'y=2')
print(Ut([1,1,2]),'z=2')

1 all ones
4 x=2
8 y=2
16 z=2


#Exercise 1

$u(x,y,z)=1+2x+3y+4z$
follow the comments

In [ ]:

#Create Geometry and Mesh 3d
#Create x vector
# Write the expression
# Make it a function
# Evaluate it and print it at [1,1,1]

#Exercise 2

$u(x,y)=1+\sin(x)*\cos(3y)$
follow the comments



In [ ]:
#Create Geometry and Mesh 2d
#Create x vector
# Write the expression
# Make it a function
# Evaluate it and print it at [1,1]




# Custom Functions


---



One thing about all the previous codes was that we begin with a UFL expression and ending with a python function that we can call. 
# e.g.

In [78]:
trial_mesh= mesh.create_unit_square(MPI.COMM_WORLD,50,50)
#2d geometry where we will work
x = ufl.SpatialCoordinate(trial_mesh)

In [79]:
Expression=sin(x[0]**3+x[1]**3)  # the UFL expression
function=lambda x: eval(str(Expression))  # the function

In [80]:
output_UFL=function([1,0])
print(output_UFL)

0.8414709848078965


Instead we can also make custom functions using numpy functions, such that they take an array as input i.e. []

In [81]:
def custom_function(x):
  return numpy.sin(x[0]**3+x[1]**3)


In [82]:
output_C=custom_function([1,0])
print(output_C)

0.8414709848078965


# So why UFL functions? 

## because they can be processed using UFL Operators like 

## $\nabla$
## $\nabla^2$

## $\nabla \times ̇$

we will use the laplacian operator with an UFL cubic expression

In [83]:
f=div(grad(Expression))
print(f)
print(type(f))

div(grad(sin(x[0] ** 3 + x[1] ** 3)))
<class 'ufl.differentiation.Div'>


# this form can be directly used in weak forms

In [84]:
V = fem.FunctionSpace(trial_mesh, ("CG", 1))
v = ufl.TestFunction(V)
F=f*v*dx
print(type(F))

<class 'ufl.form.Form'>


# The f expression can be directly used to create a forcing function

# custom functions of test function can be used in UFL as well

In [85]:
def A(u):
  return 5*u


uh=ufl.TrialFunction(V)
Bilinear=A(uh)*inner(grad(uh),grad(v))*dx
print(type(Bilinear))

<class 'ufl.form.Form'>


# Exercise: Custom Function

define a custom function to return $u^3+u^2$

In [ ]:
#define function
#return value 

# in the next workbook we will use expressions to define boundary